### Part C: Programming Task (30 marks)
#### Implement in Python:
* A* search algorithm (with admissible heuristics).
* A CSP solver for the N-Queens problem (use backtracking with either forward checking or arc consistency).


#### Requirements:
1. Print the sequence of expanded nodes for A*.
2. Display one valid solution for the N-Queens problem (e.g., for N=8).
3. Include comments and sample outputs. (30 marks)


In [1]:
import heapq

#Manhattan distance between two points
def manhattan(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def astar_grid(start, goal, blocks, w, h):
    #Check if point is within grid boundaries
    def in_bounds(p):
        r, c = p
        return 0 <= r < h and 0 <= c < w

    #Generate valid neighboring cells (up, down, left, right)
    def neighbors(p):
        r, c = p
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            q = (r+dr, c+dc)
            if in_bounds(q) and q not in blocks:
                yield q

    g = {start: 0}
    parent = {start: None}
    expanded = []
    closed = set()

    #Priority queue (min-heap) storing (f = g + h, counter, node)
    pq = []
    cnt = 0
    heapq.heappush(pq, (manhattan(start, goal), cnt, start)); cnt += 1

    #Main A* loop
    while pq:
        _, _, cur = heapq.heappop(pq)
        if cur in closed:
            continue
        expanded.append(cur)
        closed.add(cur)

        #Goal reached - reconstruct path
        if cur == goal:
            path = []
            x = cur
            while x is not None:
                path.append(x)
                x = parent[x]
            path.reverse()
            return path, expanded, g[cur]

        #Explore neighbors
        for nb in neighbors(cur):
            newg = g[cur] + 1
            if nb not in g or newg < g[nb]:
                g[nb] = newg
                parent[nb] = cur
                f = newg + manhattan(nb, goal)
                heapq.heappush(pq, (f, cnt, nb)); cnt += 1

    #No path found
    return None, expanded, float("inf")

if __name__ == "__main__":
    h, w = 5, 6
    start, goal = (0, 0), (4, 5)
    blocks = {(1,1), (1,2), (2,3)}

    path, expanded, cost = astar_grid(start, goal, blocks, w, h)

    print("Start:", start, "Goal:", goal)
    print("Expanded order:")
    for v in expanded:
        print(v)

    if path is None:
        print("\nNo path found.")
    else:
        print("\nFound path: cost =", cost, "steps =", len(path))
        print(path)


Start: (0, 0) Goal: (4, 5)
Expanded order:
(0, 0)
(1, 0)
(0, 1)
(2, 0)
(0, 2)
(3, 0)
(2, 1)
(0, 3)
(4, 0)
(3, 1)
(2, 2)
(1, 3)
(0, 4)
(4, 1)
(3, 2)
(1, 4)
(0, 5)
(4, 2)
(3, 3)
(2, 4)
(1, 5)
(4, 3)
(3, 4)
(2, 5)
(4, 4)
(3, 5)
(4, 5)

Found path: cost = 9 steps = 10
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]


In [2]:
def solve_nqueens_fc(N=8):
    #Initialize domains: each row can take any column initially
    domains = {r: set(range(N)) for r in range(N)}
    assignment = {}
    steps = {"nodes": 0}

    #Check if placing a queen is consistent with current assignment
    def consistent(row, col):
        for r2, c2 in assignment.items():
            if c2 == col or abs(c2 - col) == abs(r2 - row):
                return False
        return True

    #Apply forward checking (remove conflicting values from future domains)
    def forward_check(row, col, doms):
        removed = []
        for r2 in range(row + 1, N):
            if r2 in assignment:
                continue
            bad = set()
            for v in doms[r2]:
                if v == col or abs(v - col) == abs(r2 - row):
                    bad.add(v)
            if bad:
                doms[r2] -= bad
                for v in bad:
                    removed.append((r2, v))
            #Inconsistency detected
            if not doms[r2]:
                #Restore removed values
                for rr, vv in removed:
                    doms[rr].add(vv)
                return None
        return removed

    #Undo domain reductions after backtracking
    def undo(doms, removed):
        for r, v in removed:
            doms[r].add(v)

    #Recursive backtracking with forward checking
    def backtrack(row=0, doms=None):
        steps["nodes"] += 1
        if doms is None:
            doms = {r: set(vs) for r, vs in domains.items()}
            
        #Found a valid solution
        if row == N:
            return dict(assignment)

        for col in sorted(doms[row]):
            if not consistent(row, col):
                continue
            assignment[row] = col
            removed = forward_check(row, col, doms)
            if removed is not None:
                sol = backtrack(row + 1, doms)
                if sol is not None:
                    return sol
                undo(doms, removed)
            del assignment[row]
        return None

    solution = backtrack()
    return solution, steps["nodes"]

#Print solution
def print_board(sol):
    if sol is None:
        print("No solution.")
        return
    N = len(sol)
    print("One valid solution (row -> col):")
    print(sorted(sol.items()))
    print("\nBoard:")
    for r in range(N):
        row = ["." for _ in range(N)]
        row[sol[r]] = "Q"
        print("".join(row))

if __name__ == "__main__":
    solution, nodes = solve_nqueens_fc(8)
    print(f"Search nodes expanded: {nodes}")
    print_board(solution)


Search nodes expanded: 54
One valid solution (row -> col):
[(0, 0), (1, 4), (2, 7), (3, 5), (4, 2), (5, 6), (6, 1), (7, 3)]

Board:
Q.......
....Q...
.......Q
.....Q..
..Q.....
......Q.
.Q......
...Q....
